In [3]:
from air_hockey_challenge.framework.air_hockey_challenge_wrapper import AirHockeyChallengeWrapper

from air_hockey_challenge.environments.iiwas.env_hitting import AirHockeyHit
from air_hockey_agent.agent_builder_ddpg_hit import build_agent
import numpy as np

pybullet build time: May 20 2022 19:44:17


In [4]:
env = AirHockeyChallengeWrapper(env="7dof-hit", interpolation_order=3, debug=False)
# policy = build_agent(env.env_info)
policy = build_agent(env.env_info) 

In [152]:
env.env_info

{'table': {'length': 1.948, 'width': 1.038, 'goal_width': 0.25},
 'puck': {'radius': 0.03165},
 'mallet': {'radius': 0.04815},
 'n_agents': 2,
 'robot': {'n_joints': 7,
  'ee_desired_height': 0.1645,
  'joint_vel_limit': array([[-1.48352986, -1.48352986, -1.74532925, -1.30899694, -2.26892803,
          -2.35619449, -2.35619449],
         [ 1.48352986,  1.48352986,  1.74532925,  1.30899694,  2.26892803,
           2.35619449,  2.35619449]]),
  'joint_acc_limit': array([[-14.83529864, -14.83529864, -17.45329252, -13.08996939,
          -22.68928028, -23.5619449 , -23.5619449 ],
         [ 14.83529864,  14.83529864,  17.45329252,  13.08996939,
           22.68928028,  23.5619449 ,  23.5619449 ]]),
  'base_frame': [array([[ 1.  ,  0.  ,  0.  , -1.51],
          [ 0.  ,  1.  ,  0.  ,  0.  ],
          [ 0.  ,  0.  ,  1.  , -0.1 ],
          [ 0.  ,  0.  ,  0.  ,  1.  ]]),
   array([[-1.  ,  0.  ,  0.  ,  1.51],
          [ 0.  , -1.  ,  0.  ,  0.  ],
          [ 0.  ,  0.  ,  1.  , -0.1 ],


In [153]:
state, done = env.reset(), False
env.render()


(array([-8.59999995e-01, -1.01327613e-38,  6.45000438e-02]),
 array([0., 0., 0., 0., 0., 0.]))

(array([ 6.50000005e-01, -1.01327613e-38,  1.64500044e-01]),
 array([[-8.66025402e-01,  4.11422813e-21,  5.00000004e-01],
        [ 4.75070145e-21,  1.00000000e+00,  0.00000000e+00],
        [-5.00000004e-01,  2.37535074e-21, -8.66025402e-01]]))

In [4]:
action = policy.draw_action(np.array(state))

# Perform action
next_state, reward, done, _ = env.step(action)
env.render()

In [18]:
next_state

array([ 1.13110569e+00,  8.32590837e-02, -1.41485319e-02, -1.51341761e-01,
        7.72987041e-02, -7.04742137e-01, -8.16273550e-02, -2.06166116e-01,
        9.22085055e-02, -1.84425972e+00, -3.11343387e-02,  9.60521349e-01,
        5.59075385e-03, -2.14193421e+01, -3.23700016e-01,  1.66812944e+01,
       -2.10917481e-02,  1.07516038e+01, -1.69284319e-01, -3.49824744e+00,
        2.37373363e+00,  3.90604999e-03,  1.64211777e-01])

In [6]:
policy.get_ee_pose(next_state)

(array([ 0.6570691 , -0.01602344,  0.17382842]),
 array([[-0.8559957 , -0.03371352,  0.51588251],
        [-0.06303962,  0.99723179, -0.03943039],
        [-0.5131251 , -0.06627328, -0.85575141]]))

In [3]:
from air_hockey_challenge.utils.kinematics import inverse_kinematics
import numpy as np


In [997]:
des_pos = np.array([0.6,0.5,0.181])
state = next_state
action = np.zeros((2,7))
_,x = inverse_kinematics(policy.robot_model, policy.robot_data,des_pos)
des_v = np.array([0,1,0,0,0,0])
from air_hockey_challenge.utils.kinematics import jacobian
z = jacobian(policy.robot_model, policy.robot_data,policy.get_joint_pos(state))
z_=np.linalg.pinv(z)
v_ = des_v@z_.T
action = np.array([x,v_])
next_state, reward, done, _ = env.step(action)
env.render()
state = next_state


In [741]:
z_.T.shape

(6, 7)

In [5]:
x

array([-1.32562380e-17, -3.35883047e-02, -1.24463119e-17, -1.76999920e+00,
       -1.03464779e-17,  7.61726915e-01,  4.75070145e-21])

In [6]:
_

True

In [11]:
action = np.zeros((2,7))
action[0,:] = x[1]
action[1,:] = 0.1

In [12]:
next_state, reward, done, _ = env.step(action)
env.render()

AttributeError: 'NoneType' object has no attribute 'copy'

In [16]:
policy.get_ee_pose(next_state)

(array([0.64010856, 0.01666328, 0.1527822 ]),
 array([[-0.86484366, -0.11688435,  0.48824532],
        [-0.12211214,  0.9922888 ,  0.02124982],
        [-0.48696414, -0.04124291, -0.87244768]]))

In [ ]:
env.env_info

In [162]:
env.env_info['robot']['ee_desired_height']

0.1645

In [161]:
env._mdp_info.observation_space.shape
# env.env_info['rl_info'].action_space.low

(23,)

In [157]:
env.base_env.env_info["rl_info"].observation_space.low.shape

(23,)

In [153]:
import numpy as np
pos_max = env.env_info['robot']['joint_pos_limit'][1]
vel_max = env.env_info['robot']['joint_vel_limit'][1] 
max_ = np.stack([pos_max,vel_max])
max_action  =   max_.reshape(14,)

In [154]:
max_action

array([2.96706   , 2.0944    , 2.96706   , 2.0944    , 2.96706   ,
       2.0944    , 3.05433   , 1.48352986, 1.48352986, 1.74532925,
       1.30899694, 2.26892803, 2.35619449, 2.35619449])

In [19]:
from air_hockey_challenge.utils.transformations import world_to_robot
print(env.env_info['robot']['base_frame'][0])
x = world_to_robot(env.env_info['robot']['base_frame'][0],env.env_info['table']['length'])

[[ 1.    0.    0.   -1.51]
 [ 0.    1.    0.    0.  ]
 [ 0.    0.    1.   -0.1 ]
 [ 0.    0.    0.    1.  ]]


TypeError: object of type 'float' has no len()

In [199]:
import torch
loss = torch.nn.MSELoss(reduction='none')
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5,requires_grad=True)
output = loss(input, target)
# print(output.detach().numpy())
print(output)

tensor([[2.0921, 5.6445, 0.0062, 4.3529, 1.1709],
        [5.9289, 2.4195, 4.5575, 0.4351, 1.1163],
        [3.7259, 0.4411, 2.3772, 3.0531, 3.1126]], grad_fn=<MseLossBackward0>)


In [207]:
output.backward(torch.ones_like(input)*1.0,retain_graph=True)
print(output.grad.data)
# output.grad.data.zero_()

RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [202]:
print(output)

tensor([[2.0921, 5.6445, 0.0062, 4.3529, 1.1709],
        [5.9289, 2.4195, 4.5575, 0.4351, 1.1163],
        [3.7259, 0.4411, 2.3772, 3.0531, 3.1126]], grad_fn=<MseLossBackward0>)


In [177]:
ee_pos = policy.get_ee_pose(state)[0] 
print(ee_pos)
des_z = env.env_info['robot']['ee_desired_height']
print(des_z)

[ 6.50000005e-01 -1.01327613e-38  1.64500044e-01]
0.1645


In [6]:
from air_hockey_challenge.utils.kinematics import jacobian
z = jacobian(policy.robot_model, policy.robot_data,policy.get_joint_pos(next_state))

In [7]:
import numpy as np
z_=np.linalg.pinv(z)
z_

array([[-3.63307339e-02,  5.45445223e-01, -1.70351825e-02,
        -1.25884375e-01, -2.57123021e-02,  9.23221873e-02],
       [ 2.47507481e+00,  3.98607227e-03, -1.66459595e-01,
         1.33672356e-03,  1.35556023e+00,  7.98592369e-03],
       [ 2.07793699e-01,  2.15138731e+00, -4.42137168e-02,
        -1.37850011e+00,  1.02719018e-01, -9.65168396e-01],
       [ 1.93804388e+00,  4.01639166e-02,  2.37516167e+00,
        -2.70968806e-02,  1.92131249e+00, -1.03792897e-02],
       [-3.01003306e-01, -1.20850806e+00,  5.27324335e-03,
         1.67800605e+00, -1.75952527e-01,  9.57551849e-01],
       [-5.33100740e-01, -1.31061225e-02,  2.54146748e+00,
         4.76265578e-03,  1.56793961e+00,  6.87380612e-03],
       [ 2.55907575e-01,  3.18908998e+00, -7.51364255e-03,
        -1.85314205e+00,  1.49606133e-01, -2.23660581e+00]])

In [15]:
v = np.array([-1.0,1.0,0])

In [16]:
t = z_[:,:3]@v[:3]
print(t)

[ 0.58177596 -2.47108874  1.94359361 -1.89787997 -0.90750476  0.51999462
  2.9331824 ]


In [17]:
t1=t@z.T
t1[:3]

array([-1.00000000e+00,  1.00000000e+00,  1.11369247e-15])

In [1013]:
v@z_.T[:3,:]

array([-1.50704702,  3.85985968,  0.30996018,  6.07378655,  0.92995795,
        2.01908885, -0.97533899])

: 

In [1009]:
z_.shape

(7, 6)

In [25]:
env.base_env.is_absorbing(next_state)

True

In [27]:
policy.get_puck_pos(next_state)

array([1.01246589, 0.05187413, 0.00126406])

In [33]:
env.base_env.get_puck(next_state)

ValueError: not enough values to unpack (expected 3, got 2)

In [34]:
env.base_env.is_absorbing(next_state)

True

In [52]:
env.base_env.get_ee

<bound method AirHockeyDouble.get_ee of <air_hockey_challenge.environments.position_control_wrapper.IiwaPositionHit object at 0x7f756178b6d0>>

In [50]:
policy.get_ee_pose(next_state)[0]+[-1.51,0,-0.1]

array([-0.8531791 ,  0.00825637,  0.07359727])

In [51]:
env.base_env.vec_puck_goal

AttributeError: 'IiwaPositionHit' object has no attribute 'vec_puck_goal'

In [55]:
env.base_env.hit_range

array([[-0.7    , -0.2    ],
       [-0.39105,  0.39105]])

In [58]:
env.base_env._check_collision("puck", "robot_1/ee")

<bound method MuJoCo._check_collision of <air_hockey_challenge.environments.position_control_wrapper.IiwaPositionHit object at 0x7f756178b6d0>>

(array([-8.59999995e-01, -1.01327613e-38,  6.45000438e-02]),
 array([0., 0., 0., 0., 0., 0.]))

In [63]:
from air_hockey_challenge.utils.kinematics import jacobian
z = jacobian(policy.robot_model, policy.robot_data,policy.get_joint_pos(state))
z.shape
print("base",env.base_env.get_ee()[1])
print("policy",z@policy.get_joint_vel(state))
action = policy.draw_action(state)
next_state, reward, done, _ = env.step(action)
state = next_state
env.render()

base [ -3.18763656 -12.30253387  -3.49923488   4.59658214   4.80111548
  -5.3540546 ]
policy [  4.24343658   0.97181759   4.94453865   9.07481024 -12.54228924
  -5.43096623]


In [75]:
p_v = policy.get_joint_vel(state)
p_v

array([ -4.79115084,   3.58595677,  -3.45307492,   0.57149093,
        48.54068038,  -4.37862166, -19.12015376])

In [67]:
env.base_env.get_ee()[1]


array([ 12.09555614, -14.46712135,  -6.3617671 ,   7.5953989 ,
        12.7706603 ,   0.52782378])

In [69]:
z = jacobian(policy.robot_model, policy.robot_data,policy.get_joint_pos(state))
z.shape

(6, 7)

In [76]:
p_v_ = z@p_v
p_v_

array([  5.08926831,   4.32714412,  11.44067089,  23.38231892,
       -20.22833642,   0.23113681])

In [72]:

env.base_env.get_ee()

(array([-0.59388281, -0.22604411,  0.43446564]),
 array([ 12.09555614, -14.46712135,  -6.3617671 ,   7.5953989 ,
         12.7706603 ,   0.52782378]))

In [1056]:
from air_hockey_challenge.utils.kinematics import inverse_kinematics, jacobian
action = np.zeros((2,7))
des_pos = np.array([0.78,-0.5,0.1645])
_,x = inverse_kinematics(policy.robot_model, policy.robot_data,des_pos)
action[0,:] = x
des_v = np.array(([0,-5,0]))
jac = jacobian(policy.robot_model, policy.robot_data,policy.get_joint_pos(state))
inv_jac = np.linalg.pinv(jac)
joint_vel = des_v@inv_jac.T[:3,:]
action[1,:] = joint_vel
next_state, reward, done, _ = env.step(action)
state = next_state
env.render()
print("base",env.base_env.get_ee()[1])
print("policy",(jac@policy.get_joint_vel(state))[:])


base [ 2.35882643  0.07925349  5.2474227  -0.497722   -3.95492933  1.77396032]
policy [ 1.54583013 -0.84592212  0.71987076 -1.77999328 -0.6070855   3.67315489]


In [940]:
env.reset()

array([ 9.72749827e-01,  2.36097889e-01,  0.00000000e+00, -3.38691158e-01,
       -4.79851110e-02,  4.22380038e-01,  4.02979867e-22, -1.96067345e-01,
        2.56756848e-21, -1.84363898e+00, -1.86530826e-21,  9.70422238e-01,
        4.75070145e-21,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        2.37000000e+00,  1.01327613e-38,  1.64500044e-01])

In [1058]:
policy.get_ee_pose(next_state)[0][:2]

array([ 0.40207824, -0.50333883])

: 

In [6]:
env.base_env.hit_range

array([[-0.7    , -0.2    ],
       [-0.39105,  0.39105]])